In [1]:
%reload_ext autoreload
%autoreload 2

from src.acnets.deep import WaveletModel, MaskedModel
from src.acnets.datasets import LEMONDataModule, Julia2018DataModule


lemon_datamodule = LEMONDataModule(
    atlas='aal', kind='partial correlation',
    test_ratio=.1, val_ratio=.05,
    n_subjects=203, batch_size=16)
lemon_datamodule.setup()

julia2018_datamodule = Julia2018DataModule(
    atlas='aal', kind='partial correlation',
    test_ratio=.25, batch_size=32)
julia2018_datamodule.setup()

n_regions = lemon_datamodule.train[0][0].shape[1]
n_wavelets = 124
n_embeddings = 8
segment_length = 32

model = MaskedModel(n_regions, n_embeddings, segment_length)

trainer = model.fit(lemon_datamodule, max_epochs=1000, phase='pretrain')
tuner = model.fit(julia2018_datamodule, max_epochs=5000, phase='finetune')

FileNotFoundError: [Errno 2] No such file or directory: 'data/atlases/AAL3v1.csv'